In [2]:
import logging

from pathlib import Path

from Pegasus.api import *

logging.basicConfig(level=logging.DEBUG)

# Properties
props = Properties()
props["pegasus.data.configuration"] = "condorio"
props["dagman.retry"] = "2"
props.write()

# Replica
'''
rc = ReplicaCatalog()\
        .add_replica("local", "saved_state.txt", Path(".").resolve() / "saved_state.txt")\
        .write()
'''

# Transformation
exe = Transformation(
        "checkpoint_program.py",
        site="local",
        pfn=str(Path(".").resolve() / "checkpoint_program.py"),
        is_stageable=True
    )

tc = TransformationCatalog().add_transformations(exe).write()

# Workflow
job = Job(exe)\
        .add_args(180)\
        .add_checkpoint(File("saved_state.txt"), stage_out=True)\
        .set_stdout("output.txt")\
        .add_profiles(Namespace.PEGASUS, key="checkpoint.time", value=1)\
        .add_profiles(Namespace.PEGASUS, key="maxwalltime", value=2)

'''
KILL signal is sent at (checkpoint.time + (maxwalltime-checkpoint.time)/2) minutes. (hence -K 30)
.add_profiles(Namespace.PEGASUS, key="checkpoint.time", value=1)\
.add_profiles(Namespace.PEGASUS, key="maxwalltime", value=2)
# Kickstart args (note checkpoint.time converted to seconds)
pegasus-kickstart  -n checkpoint_program.py -N ID0000001 -o output.txt -R condorpool  -s output.txt=output.txt -s saved_state.txt=saved_state.txt -L checkpoint-wf -T 2020-07-07T05:35:37+00:00 -k 60 -K 30 ./checkpoint_program_py
'''

wf = Workflow("checkpoint-wf").add_jobs(job).add_metadata(metadata_key="value")

try:
    wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)

Plan:
[main] WARN  schema.JsonMetaSchema  - Unknown keyword $defs - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword additionalItems - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
[main] WARN  schema.JsonMetaSchema  - Unknown keyword examples - you should define your own Meta Schema. If the keyword is irrelevant for validation, just use a NonValidationKeyword
2020.07.16 00:21:39.376 UTC:    
2020.07.16 00:21:39.383 UTC:   ----------------------------------------------------------------------- 
2020.07.16 00:21:39.388 UTC:   File for submitting this DAG to HTCondor           : checkpoint-wf-0.dag.condor.sub 
2020.07.16 00:21:39.394 UTC:   Log of DAGMan debugging messages                 : checkpoint-wf-0.dag.dagman.out 
2020.07.16 00:21:39.400 UTC:   Log of HTCondor library output                

[##################################################] 100.0% ..Success (Completed: 9, Queued: 0, Running: 0, Failed: 0)


Analyzer:
Your database is compatible with Pegasus version: 5.0.0dev

************************************Summary*************************************

 Submit Directory   : /home/scitech/shared-data/sample-checkpoint-condorio-wf/scitech/pegasus/checkpoint-wf/run0003
 Total jobs         :      9 (100.00%)
 # jobs succeeded   :      9 (100.00%)
 # jobs failed      :      0 (0.00%)
 # jobs held        :      0 (0.00%)
 # jobs unsubmitted :      0 (0.00%)

 

Statistics:
Your database is compatible with Pegasus version: 5.0.0dev

#
# Pegasus Workflow Management System - http://pegasus.isi.edu
#
# Workflow summary:
#   Summary of the workflow execution. It shows total
#   tasks/jobs/sub workflows run, how many succeeded/failed etc.
#   In case of hierarchical workflow the calculation shows the
#   statistics across all the sub workflows.It shows the following
#   statistics about tasks, jobs and sub workflows.
#     * Succeeded - total count of succeeded tasks/jobs/sub workflows.
#     * F